In [ ]:

import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
     sys.path.insert(0, project_root)

import json
from dotenv import load_dotenv
from supabase import create_client, Client
from google.genai import types
import traceback
import time

from src.llm.tools.ChunkRetriever import RetrievalService
from src.llm.OpenAIClient import OpenAIClient
from src.llm.GeminiClient import GeminiClient
from src.storage.SupabaseService import SupabaseService
from src.helper.llm_helper_chat import create_final_answer_instructions, print_final_formatted_answer, serialize_conversation_history, format_chunks_for_llm


# --- Configuration Constants ---
YOUR_APP_DOMAIN = "www.stackifier.com"

# --- SETUP FUNCTION ---
def initialize_clients_and_authenticate_for_test() -> tuple[OpenAIClient, GeminiClient, SupabaseService, str, Client, RetrievalService]:
    """
    Loads .env, initializes clients (OpenAI, Gemini, Supabase),
    authenticates the test user, initializes RetrievalService,
    and returns the clients, user ID, and services.
    """
    load_dotenv()

    TEST_EMAIL = os.environ.get("TEST_EMAIL")
    TEST_PASSWORD = os.environ.get("TEST_PASSWORD")
    if not TEST_EMAIL or not TEST_PASSWORD:
        raise ValueError("TEST_EMAIL and TEST_PASSWORD must be set in your .env file.")

    try:
        print("\n--- Initializing clients, authenticating, and setting up services ---")
        # Initialize dependent clients first
        openai_client_local = OpenAIClient()
        gemini_client_local = GeminiClient() 

        supabase_url = os.environ.get("SUPABASE_URL")
        supabase_key = os.environ.get("SUPABASE_ANON_KEY")
        if not supabase_url or not supabase_key:
            raise ValueError("SUPABASE_URL and SUPABASE_ANON_KEY must be set in your .env file.")

        # Authenticate Supabase client
        auth_client_local = create_client(supabase_url, supabase_key)
        print("Supabase client created.")

        print(f"Attempting to sign in with email: {TEST_EMAIL}")
        auth_response = auth_client_local.auth.sign_in_with_password(
            {"email": TEST_EMAIL, "password": TEST_PASSWORD}
        )

        if not auth_response or not auth_response.user:
            error_detail = auth_response.error.message if hasattr(auth_response, 'error') and auth_response.error else "Unknown authentication error"
            raise ConnectionError(f"Supabase authentication failed: {error_detail}. Check credentials and Supabase Auth settings.")

        authenticated_user_id_str_local = str(auth_response.user.id)
        print(f"Authentication successful. User ID: {authenticated_user_id_str_local}")

        # Initialize Services using the clients
        supabase_service_local = SupabaseService(supabase_client=auth_client_local)
        # MODIFIED: Initialize RetrievalService
        retrieval_service_local = RetrievalService(
            openai_client=openai_client_local,
            supabase_service=supabase_service_local
        )

        print("Clients and Services initialized and authenticated.")

        # MODIFIED: Return the new service instance
        return openai_client_local, gemini_client_local, supabase_service_local, authenticated_user_id_str_local, auth_client_local, retrieval_service_local

    except Exception as e:
        print(f"Initialization or Authentication Error: {str(e)}\n{traceback.format_exc()}")
        sys.exit(1)

# --- MAIN QUERY PROCESSING PIPELINE FUNCTION ---
def run_financial_query_pipeline(
    user_query_text: str,
    initial_conv_history: list,
    gemini_client_instance: GeminiClient,
    auth_user_id: str,
    tool_definition: types.Tool,
    model_name_to_use: str,
    retrieval_service_instance: RetrievalService
) -> tuple[str, list]:
    """
    Manages the multi-turn conversation with Gemini to answer a financial query,
    potentially using the retrieve_financial_chunks tool via the RetrievalService.
    Returns the final answer text and the updated conversation history.
    """
    current_conv_history = list(initial_conv_history)

    try:
        # 1. First call to LLM: Send user query and retrieval tool definition
        print(f"\n--- Sending initial query to Gemini ({model_name_to_use}) ---")
        current_conv_history.append(types.Content(role="user", parts=[types.Part(text=user_query_text)]))
        print(f"  Conversation History before first call:\n{json.dumps(serialize_conversation_history(current_conv_history), indent=2)}")

        response = gemini_client_instance.client.models.generate_content(
            model=model_name_to_use,
            contents=current_conv_history,
            config=types.GenerateContentConfig(
                tools=[tool_definition],
                automatic_function_calling= {"disable": True},
                tool_config= {"function_calling_config": {"mode": "any"}},
                temperature=0
                )
        )
        print(f"\n--- Received response from first Gemini call ---")

        if not response.candidates or not response.candidates[0].content or not response.candidates[0].content.parts:
            error_msg = "Error: Unexpected response structure or no candidates/parts from Gemini's first call."
            print(error_msg)
            if hasattr(response, 'prompt_feedback') and response.prompt_feedback: print(f"Prompt Feedback: {response.prompt_feedback}")
            return error_msg, current_conv_history

        model_response_content = response.candidates[0].content
        message_part = model_response_content.parts[0]
        current_conv_history.append(model_response_content)
        print(f"\n  Model's response content (first call) added to history.")

        # 2. Check if LLM requested a function call
        if hasattr(message_part, 'function_call') and message_part.function_call:
            function_call = message_part.function_call
            print(f"\n--- Gemini requested function call: '{function_call.name}' ---")
            tool_args = dict(function_call.args)
            print(f"  Raw Arguments from LLM: {tool_args}")

            if function_call.name == "retrieve_financial_chunks":
                print("  Recognized 'retrieve_financial_chunks' call.")
                
                # Pass the user_id and LLM provided args, but NOT the clients
                function_result_json = retrieval_service_instance.retrieve_chunks(
                    user_id=auth_user_id, # Pass user_id explicitly
                    **tool_args # Unpack the args provided by the LLM
                )

                print(f"\n--- Finished executing RetrievalService.retrieve_chunks ---")
                print(f"  Function result (JSON string, first 500 chars):\n{function_result_json[:500]}...")

                print("\n--- Preparing enriched context and instructions for final Gemini call ---")
                try:
                    function_response_data = json.loads(function_result_json)
                except json.JSONDecodeError:
                    print(f"Error decoding function result JSON: {function_result_json[:200]}...")
                    function_response_data = {"error": "Invalid JSON from tool."}

                function_response_part = types.Part.from_function_response(
                    name=function_call.name,
                    response={"result": function_response_data}
                )
                current_conv_history.append(types.Content(role="user", parts=[function_response_part]))
                print(f"  Raw function response part added to history.")

                # Assuming format_chunks_for_llm can handle the JSON string output
                formatted_snippets_text = format_chunks_for_llm(function_result_json)
                final_instructions_text = create_final_answer_instructions(user_query_text, formatted_snippets_text, YOUR_APP_DOMAIN)
                current_conv_history.append(types.Content(role="user", parts=[types.Part(text=final_instructions_text)]))
                print(f"  Formatted snippets and citation instructions added to history.")
                print(f"\n  Conversation History before second call (final answer generation):\n{json.dumps(serialize_conversation_history(current_conv_history), indent=2)}")

                final_response = gemini_client_instance.client.models.generate_content(
                    model=model_name_to_use,
                    contents=current_conv_history
                )
                print(f"\n--- Received response from second Gemini call (Final Answer) ---")

                if final_response.candidates and final_response.candidates[0].content and final_response.candidates[0].content.parts:
                    final_model_response_content = final_response.candidates[0].content
                    current_conv_history.append(final_model_response_content)
                    return final_response.text, current_conv_history
                else:
                    error_msg = "Error: No final response text found after sending function result."
                    print(error_msg)
                    if hasattr(final_response, 'prompt_feedback') and final_response.prompt_feedback: print(f"Final Response Prompt Feedback: {final_response.prompt_feedback}")
                    return error_msg, current_conv_history
            else:
                warning_msg = f"Warning: LLM requested unknown function '{function_call.name}'. Stopping execution due to unknown function call."
                print(warning_msg)
                return warning_msg, current_conv_history
        else:
            print("\n--- Gemini decided to answer directly (No Function Call Requested) ---")
            if hasattr(message_part, 'text') and message_part.text is not None:
                print(message_part.text)
                return message_part.text, current_conv_history
            else:
                no_text_msg = "No text response found in the initial call and no function call made. Stopping execution after direct answer or unexpected initial response."
                print(no_text_msg)
                return no_text_msg, current_conv_history
    except Exception as e:
        error_msg = f"\nAn unexpected error occurred during the Gemini interaction: {str(e)}\n{traceback.format_exc()}"
        print(error_msg)
        return error_msg, current_conv_history


# --- MAIN EXECUTION ---
if __name__ == "__main__":
    # Initalize all variables and services
    openai_client_main, gemini_client_main, supabase_service_main, authenticated_user_id_str_main, auth_client_main, retrieval_service_main = initialize_clients_and_authenticate_for_test()
    
    # User Query and Conversation History
    t1 = time.time()
    user_query_main = "Whats the Gross Carrying Amount for Total intangible assets for tesla in 2021? Create a report of tesla for 2021 in markdown i can copy."
    conversation_history_main = [] # Initialize fresh for each run, or load if continuing

    print(f"\n--- User Query ---")
    print(user_query_main)

    # Tool and Model Configuration
    retrieval_tool_main = RetrievalService.get_tool_declaration()
    gemini_model_name_main = "gemini-2.0-flash"

    # Run the main processing pipeline
    final_answer_text, updated_history = run_financial_query_pipeline(
        user_query_text=user_query_main,
        initial_conv_history=conversation_history_main,
        gemini_client_instance=gemini_client_main, 
        auth_user_id=authenticated_user_id_str_main,
        tool_definition=retrieval_tool_main,
        model_name_to_use=gemini_model_name_main,
        retrieval_service_instance=retrieval_service_main
    )
    t2 = time.time()

    # Update conversation history if you plan to continue the conversation
    conversation_history_main = updated_history

    # Optionally, print the full conversation history for debugging
    print("\n--- Full Conversation History (Serialized) ---")
    print(json.dumps(serialize_conversation_history(conversation_history_main), indent=2))
    print(f"\n[TIMER] TOTAL ELAPSED: {(t2 - t1):.2f}s")
    
    # Print the final answer
    print_final_formatted_answer(final_answer_text)